In [8]:
# ------------------------------------------------------
# Holiday in Romania Problem
# Solved with:
# 1. Uniform Cost Search (UCS)
# 2. Greedy Best-First Search (GBFS)
# 3. A* Search
# ------------------------------------------------------

from heapq import heappush, heappop   # priority queue for frontier

# Restore built-in print (in case it was overwritten earlier in your notebook)
import builtins
print = builtins.print

# ------------------------------------------------------
# Graph of Romania (road map with distances)
# ------------------------------------------------------
graph = {
    "Arad": {"Zerind": 75, "Sibiu": 140, "Timisoara": 118},
    "Zerind": {"Arad": 75, "Oradea": 71},
    "Oradea": {"Zerind": 71, "Sibiu": 151},
    "Sibiu": {"Arad": 140, "Oradea": 151, "Fagaras": 99, "Rimnicu Vilcea": 80},
    "Timisoara": {"Arad": 118, "Lugoj": 111},
    "Lugoj": {"Timisoara": 111, "Mehadia": 70},
    "Mehadia": {"Lugoj": 70, "Dobreta": 75},
    "Dobreta": {"Mehadia": 75, "Craiova": 120},
    "Craiova": {"Dobreta": 120, "Rimnicu Vilcea": 146, "Pitesti": 138},
    "Rimnicu Vilcea": {"Sibiu": 80, "Craiova": 146, "Pitesti": 97},
    "Fagaras": {"Sibiu": 99, "Bucharest": 211},
    "Pitesti": {"Rimnicu Vilcea": 97, "Craiova": 138, "Bucharest": 101},
    "Bucharest": {"Fagaras": 211, "Pitesti": 101, "Giurgiu": 90, "Urziceni": 85},
    "Giurgiu": {"Bucharest": 90},
    "Urziceni": {"Bucharest": 85, "Hirsova": 98, "Vaslui": 142},
    "Hirsova": {"Urziceni": 98, "Eforie": 86},
    "Eforie": {"Hirsova": 86},
    "Vaslui": {"Urziceni": 142, "Iasi": 92},
    "Iasi": {"Vaslui": 92, "Neamt": 87},
    "Neamt": {"Iasi": 87}
}

# ------------------------------------------------------
# Straight-line distance heuristic (h) to Bucharest
# (from Artificial Intelligence: A Modern Approach)
# ------------------------------------------------------
h = {
    "Arad": 366, "Bucharest": 0, "Craiova": 160, "Dobreta": 242, "Eforie": 161,
    "Fagaras": 176, "Giurgiu": 77, "Hirsova": 151, "Iasi": 226, "Lugoj": 244,
    "Mehadia": 241, "Neamt": 234, "Oradea": 380, "Pitesti": 100, "Rimnicu Vilcea": 193,
    "Sibiu": 253, "Timisoara": 329, "Urziceni": 80, "Vaslui": 199, "Zerind": 374
}

# ------------------------------------------------------
# Helper function: Reconstruct the path from start to goal
# using the 'parent' dictionary
# ------------------------------------------------------
def reconstruct_path(parent, goal):
    path = []
    node = goal
    while node is not None:     # walk backwards until start
        path.append(node)
        node = parent.get(node)
    return list(reversed(path)) # reverse to get start → goal

# ------------------------------------------------------
# Uniform Cost Search (like Dijkstra’s algorithm)
# Expands nodes in order of lowest path cost g(n)
# ------------------------------------------------------
def ucs(start, goal):
    frontier = []                          # priority queue (min-heap)
    heappush(frontier, (0, start))         # (cost so far, city)
    parent = {start: None}                 # store path backpointers
    best_g = {start: 0}                    # best cost to each city
    expanded_order, closed = [], set()     # keep track of expansions

    while frontier:
        g, current = heappop(frontier)     # node with lowest g(n)
        if current in closed:
            continue
        closed.add(current)
        expanded_order.append(current)

        if current == goal:                # goal found
            return {
                "path": reconstruct_path(parent, goal),
                "cost": g,
                "expanded": len(expanded_order),
                "expanded_order": expanded_order
            }

        # explore neighbors
        for nxt, w in graph[current].items():
            ng = g + w                     # new path cost
            if nxt not in best_g or ng < best_g[nxt]:
                best_g[nxt] = ng
                parent[nxt] = current
                heappush(frontier, (ng, nxt))
    return None

# ------------------------------------------------------
# Greedy Best-First Search
# Expands nodes in order of heuristic h(n) only
# Fast, but not guaranteed optimal
# ------------------------------------------------------
def gbfs(start, goal):
    frontier = []
    heappush(frontier, (h[start], start))  # prioritize by h(n)
    parent = {start: None}
    visited = set()
    expanded_order = []

    while frontier:
        _, current = heappop(frontier)
        if current in visited:
            continue
        visited.add(current)
        expanded_order.append(current)

        if current == goal:                # goal found
            path = reconstruct_path(parent, goal)
            cost = sum(graph[a][b] for a, b in zip(path, path[1:])) # compute real cost
            return {
                "path": path,
                "cost": cost,
                "expanded": len(expanded_order),
                "expanded_order": expanded_order
            }

        # expand neighbors (based only on h)
        for nxt in graph[current]:
            if nxt not in visited:
                if nxt not in parent:      # record first parent
                    parent[nxt] = current
                heappush(frontier, (h[nxt], nxt))
    return None

# ------------------------------------------------------
# A* Search
# Expands nodes in order of f(n) = g(n) + h(n)
# Combines UCS and Greedy → optimal and efficient (if h admissible)
# ------------------------------------------------------
def astar(start, goal):
    frontier = []
    heappush(frontier, (h[start], 0, start)) # (f, g, city)
    parent = {start: None}
    best_g = {start: 0}
    expanded_order, closed = [], set()

    while frontier:
        f, g, current = heappop(frontier)  # pick node with smallest f(n)
        if current in closed:
            continue
        closed.add(current)
        expanded_order.append(current)

        if current == goal:                # goal found
            return {
                "path": reconstruct_path(parent, goal),
                "cost": g,
                "expanded": len(expanded_order),
                "expanded_order": expanded_order
            }

        # explore neighbors
        for nxt, w in graph[current].items():
            ng = g + w                     # new g(n)
            if nxt not in best_g or ng < best_g[nxt]:
                best_g[nxt] = ng
                parent[nxt] = current
                heappush(frontier, (ng + h[nxt], ng, nxt)) # f = g+h
    return None

# ------------------------------------------------------
# Pretty print results in readable format
# ------------------------------------------------------
def pretty_print(name, result):
    if not result:
        print(f"{name}: No solution.")
        return
    print(f"\n{name}")
    print("-" * len(name))
    print("Path: " + " -> ".join(result["path"]))
    print(f"Total cost: {result['cost']}")
    print(f"Nodes expanded: {result['expanded']}")
    print("Expansion order: " + ", ".join(result["expanded_order"]))

# ------------------------------------------------------
# Main driver
# ------------------------------------------------------
if __name__ == "__main__":
    start, goal = "Arad", "Bucharest"

    pretty_print("Uniform Cost Search (UCS)", ucs(start, goal))
    pretty_print("Greedy Best-First Search (GBFS)", gbfs(start, goal))
    pretty_print("A* Search", astar(start, goal))



Uniform Cost Search (UCS)
-------------------------
Path: Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest
Total cost: 418
Nodes expanded: 13
Expansion order: Arad, Zerind, Timisoara, Sibiu, Oradea, Rimnicu Vilcea, Lugoj, Fagaras, Mehadia, Pitesti, Craiova, Dobreta, Bucharest

Greedy Best-First Search (GBFS)
-------------------------------
Path: Arad -> Sibiu -> Fagaras -> Bucharest
Total cost: 450
Nodes expanded: 4
Expansion order: Arad, Sibiu, Fagaras, Bucharest

A* Search
---------
Path: Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest
Total cost: 418
Nodes expanded: 6
Expansion order: Arad, Sibiu, Rimnicu Vilcea, Fagaras, Pitesti, Bucharest
